In [1]:
!pip install scikit-surprise
        
from google.colab import drive
drive.mount('/content/drive')

%cd /content/drive/My Drive/Colab Notebooks

     |████████████████████████████████| 6.5MB 8.1MB/s 
  Created wheel for scikit-surprise: filename=scikit_surprise-1.1.0-cp36-cp36m-linux_x86_64.whl size=1678029 sha256=e02e8715c8e9765c3223d91db2a21212392dd5658f70129815ef7ca2fda08380
  Stored in directory: /root/.cache/pip/wheels/cc/fa/8c/16c93fccce688ae1bde7d979ff102f7bee980d9cfeb8641bcf
Successfully built scikit-surprise
Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive
/content/drive/My Drive/Colab Notebooks


In [0]:
import surprise
import importlib
import pandas as pd
import numpy as np

import source.recommender as rcm
import source.datasets as ds
import source.utils as ut
import source.forecast as ft
import source.evaluation as evl
import source.pytorchSR as pytorchSR

pd.options.mode.chained_assignment = None 
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

In [0]:
importlib.reload(rcm)
importlib.reload(ft)
importlib.reload(ds)
importlib.reload(pytorchSR)
importlib.reload(ut)

<module 'source.utils' from '/content/drive/My Drive/Colab Notebooks/source/utils.py'>

# Forecasting in Recommender Systems

In this notebook I'll try use some simple ideas of forecasting ratings in recommender systems. I'll use MovieLens 20M dataset in all models.

# Quick summary


## Movie rating forecasting

### Model

For few reasons I've chosen Last Sample as forecasting model in my algorithm:
1. The time series of movie ratings don't have any significant seasonality or clear trend.
2. No model gave better accuracy in expermients of forecasting mean rating
3. Last Sample model is the simplest and the fastest model
4. In real recommender systems we almost never want to make recommendation for the long future. Typically system can use ratings made few moments before making a new recommendation

### Method

In my approach, I create a time series of mean rating for each popular movie in dataset (popular movie is a one in a top 2/3 of movies with most number of ratings). To do this, ratings of each popular movie are gathered into 10-day timebins and then mean of each timebin is computed. This is our raw time series which is further smoothed with exponential weighted smoothing. The smoothing also includes throwing away time points with no sufficient samples. Then this smoothed time series are used to fit Last Sample models for each popular movie

## Using forecasting in recommender system

### Idea #1

In the first idea, recommender system will be used to predict deviation from mean rating of a movie instead of predicting rating explicitly. To do this, each rating value will be converted to deviation from mean rating of given movie. Then these deviations will be used as input to base recommender system algorithm. The mean of each movie will be forecasted and deviation will be computed with this forecast.


### Idea #2

I found interesting approach with time-aware recommendation: 
https://www.cse.iitk.ac.in/users/nsrivast/HCC/Recommender_systems_handbook.pdf
5.3.3 Time-aware factor model

The idea is to add new bias parameter for each timebin in dataset. So the prediction for movie is R_ui + b_i(t)

## My Neural Network algorithm

To test my approaches in a fast and flexible way, I implemented my own system recommender algorithm. This algorithm is implemented as Neural Network in PyTorch, which allows fast computations with CUDA. The main idea is based on Word2Vec algortihm. Each user and item has its embedding vector. When we want to predict rating made by user u to item i, we compute average vector of this pair and then use this new vector as input to a neural network. Thanks to PyTorch, we can use all neural network techniques like dropout or weight decay.



# Results summary

Better results are bolded in the table

## **Cross validation split method**

Cross-validation on randomly shuffled data. This is the simplest and most popular way of splitting data to test a model. This method splits data randomly into \( k \) equal parts and uses one of them as test part while remaining parts are used as train part. This procedure is repeated \( k \) times, with different part used as test data each time.


<table>
  <tr>
    <th rowspan=2>Model</th>
    <th colspan=2>RMSE (less is better)</th>
    <th colspan=2>MAE (less is better)</th>
    <th colspan=2>NDPM (less is better)</th>
    <th colspan=2>FCP (more is better)</th>
  </tr>
  <tr>
    <th>without forecasting</th>
    <th>with forecasting</th>
    <th>without forecasting</th>
    <th>with forecasting</th>
    <th>without forecasting</th>
    <th>with forecasting</th>
    <th>without forecasting</th>
    <th>with forecasting</th>
  </tr>
  
  <tr>
    <th>Baseline</th>
    <td>0.859</td>
    <td><b>0.856</b></td>
    <td>0.660</td>
    <td><b>0.658</b></td>
    <td>0.316</td>
    <td><b>0.312</b></td>
    <td>0.711</td>
    <td><b>0.712</b></td>
  </tr>
  
  <tr>
    <th>SVD</th>
    <td>0.811</td>
    <td><b>0.802</b></td>
    <td>0.624</td>
    <td><b>0.613</b></td>
    <td>0.296</td>
    <td><b>0.286</b></td>
    <td>0.744</td>
    <td><b>0.748</b></td>
  </tr>
  
  <tr>
    <th>Probabilistic Matrix Factorization</th>
    <td>0.837</td>
    <td><b>0.828</b></td>
    <td>0.648</td>
    <td><b>0.638</b></td>
    <td>0.298</td>
    <td><b>0.289</b></td>
    <td>0.735</td>
    <td><b>0.737</b></td>
  
  <tr>
    <th>Neural network based</th>
    <td>0.810</td>
    <td><b>0.809</b></td>
    <td>0.619</td>
    <td><b>0.618</b></td>
    <td>0.286</td>
    <td><b>0.284</b></td>
    <td>0.7448</td>
    <td><b>0.745</b></td>
  </tr>
</table>


## Split by time point method

Splitting data by some time point \( t \). All ratings made before \( t \) are used as train part, while all newer ratings are used as test part. This method simulates case where we have some historical data and we want to use it in new recommender system.
<table>
  <tr>
    <th rowspan=2>Model</th>
    <th colspan=2>RMSE (less is better)</th>
    <th colspan=2>MAE (less is better)</th>
    <th colspan=2>NDPM (less is better)</th>
    <th colspan=2>FCP (more is better)</th>
  </tr>
  <tr>
    <th>without forecasting</th>
    <th>with forecasting</th>
    <th>without forecasting</th>
    <th>with forecasting</th>
    <th>without forecasting</th>
    <th>with forecasting</th>
    <th>without forecasting</th>
    <th>with forecasting</th>
  </tr>
  
  <tr>
    <th>Baseline</th>
    <td>0.942</td>
    <td><b>0.936</b></td>
    <td><b>0.713</b></td>
    <td>0.715</td>
    <td>0.316</td>
    <td><b>0.312</b></td>
    <td>0.711</td>
    <td><b>0.712</b></td>
  </tr>
  
  <tr>
    <th>SVD</th>
    <td><b>0.931</b></td>
    <td>0.932</td>
    <td>0.714</td>
    <td><b>0.711</b></td>
    <td><b>0.352</b></td>
    <td>0.353</td>
    <td>0.627</td>
    <td><b>0.628</b></td>
  </tr>
  
  <tr>
    <th>Probabilistic Matrix Factorization</th>
    <td>1.010</td>
    <td><b>0.943</b></td>
    <td>0.778</td>
    <td><b>0.719</b></td>
    <td>0.484</td>
    <td><b>0.355</b></td>
    <td>0.410</td>
    <td><b>0.623</b></td>
  
  <tr>
    <th>Neural network based</th>
    <td>0.955</td>
    <td><b>0.933</b></td>
    <td>0.723</td>
    <td><b>0.713</b></td>
    <td>0.372</td>
    <td><b>0.353</b></td>
    <td>0.610</td>
    <td><b>0.627</b></td>
  </tr>
</table>

## Popularity decay split method

In this method, we group ratings by movie and throw away 80 \% of ratings done later than 3/10 of given movie lifetime. For example, if movie \( m \) oldest rating was made in 2000, and the newest in 2010, then we throw away 80 \% randomly chosen ratings made in year range 2003-2010. This simulates datasets with high and fast decrease of products' popularity.
<table>
  <tr>
    <th rowspan=2>Model</th>
    <th colspan=2>RMSE (less is better)</th>
    <th colspan=2>MAE (less is better)</th>
    <th colspan=2>NDPM (less is better)</th>
    <th colspan=2>FCP (more is better)</th>
  </tr>
  <tr>
    <th>without forecasting</th>
    <th>with forecasting</th>
    <th>without forecasting</th>
    <th>with forecasting</th>
    <th>without forecasting</th>
    <th>with forecasting</th>
    <th>without forecasting</th>
    <th>with forecasting</th>
  </tr>
  
  <tr>
    <th>Baseline</th>
    <td>0.856</td>
    <td><b>0.851</b></td>
    <td>0.643</td>
    <td><b>0.642</b></td>
    <td>0.328</td>
    <td><b>0.323</b></td>
    <td>0.662</td>
    <td><b>0.665</b></td>
  </tr>
  
  <tr>
    <th>SVD</th>
    <td><b>0.8178</b></td>
    <td>0.8179</td>
    <td><b>0.6139</b></td>
    <td>0.6143</td>
    <td>0.3116</td>
    <td><b>0.3094</b></td>
    <td>0.6818</td>
    <td><b>0.6823</b></td>
  </tr>
  
  <tr>
    <th>Probabilistic Matrix Factorization</th>
    <td><b>0.855</b></td>
    <td>0.879</td>
    <td><b>0.634</b></td>
    <td>0.650</td>
    <td><b>0.308</b></td>
    <td>0.310</td>
    <td><b>0.681</b></td>
    <td>0.673</td>
  
  <tr>
    <th>Neural network based</th>
    <td>0.818</td>
    <td><b>0.816</b></td>
    <td>0.615</td>
    <td><b>0.613</b></td>
    <td>0.306</td>
    <td><b>0.304</b></td>
    <td>0.686</td>
    <td><b>0.687</b></td>
  </tr>
</table>

## Cross validation with idea #3

<table>
  <tr>
    <th rowspan=2>Model</th>
    <th colspan=2>RMSE (less is better)</th>
    <th colspan=2>MAE (less is better)</th>
    <th colspan=2>NDPM (less is better)</th>
    <th colspan=2>FCP (more is better)</th>
  </tr>
  <tr>
    <th>without time context</th>
    <th>with time context</th>
    <th>without time context</th>
    <th>with time context</th>
    <th>without time context</th>
    <th>with time context</th>
    <th>without time context</th>
    <th>with time context</th>
  </tr>
 
  <tr>
    <th>Neural network based</th>
    <td>0.80996</td>
    <td>0.80864</td>
    <td>0.61969</td>
    <td><b>0.61913</b></td>
    <td>0.0.28616</td>
    <td><b>0.28375</b></td>
    <td>0.74430</td>
    <td><b>0.74339</b></td>
  </tr>
</table>

# Evaluation metrics
## NDPM - Normalized Distance-based Performance Measure

This measure evaluates preserved order between ranked items. It doesn't rely on exact values of predictions but only on their correct order. This measue doesn't penalize algorithm that changes order of items that have tie in reference ranking (for example when item A and item B have both rating 4.0 and our system predicted 3.5 and 4.5)


## FCP - Fraction of Concordant Pairs

Similar to NDPM:
https://www.ijcai.org/Proceedings/13/Papers/449.pdf

In [0]:
movielens20m = ds.get_movielens('20m')

In [0]:
create_cached_cross_validation_data(movielens20m, filename="cross3k_3000movies", k=3)

# Cross validation split method

## Neural Network algorithm

### Without forecasting

In [0]:
def nn_model_constructor(train_df, test_df, forecaster):
  return pytorchSR.NeuralNetworkSR(train_df, test_df, 20, 20, [256], 0.0, lr=3*1e-3, wd=1e-1, max_time=440.0, batch_size=10000, log_freq=1000)
  
ut.cross_validate_cached(nn_model_constructor, ['cross3k_3000movies0', 'cross3k_3000movies1', 'cross3k_3000movies2'])

[1,   500] loss: 1.691
[1,  1000] loss: 0.897
[2,   500] loss: 0.796
[2,  1000] loss: 0.781
[3,   500] loss: 0.746
[3,  1000] loss: 0.737
[4,   500] loss: 0.716
[4,  1000] loss: 0.717
[5,   500] loss: 0.697
[5,  1000] loss: 0.701
[6,   500] loss: 0.681
[6,  1000] loss: 0.690
[7,   500] loss: 0.671
[7,  1000] loss: 0.684
[8,   500] loss: 0.665
[8,  1000] loss: 0.680
[9,   500] loss: 0.660
[9,  1000] loss: 0.675
[10,   500] loss: 0.655
[10,  1000] loss: 0.671
[11,   500] loss: 0.651
[11,  1000] loss: 0.668
[12,   500] loss: 0.648
[12,  1000] loss: 0.665
[13,   500] loss: 0.646
[13,  1000] loss: 0.663
[14,   500] loss: 0.643
[14,  1000] loss: 0.662
[15,   500] loss: 0.642
[15,  1000] loss: 0.660
[16,   500] loss: 0.641
[16,  1000] loss: 0.660
[17,   500] loss: 0.639
[17,  1000] loss: 0.659
[18,   500] loss: 0.638
[18,  1000] loss: 0.658
[19,   500] loss: 0.638
[19,  1000] loss: 0.658
[20,   500] loss: 0.637
[20,  1000] loss: 0.657
Finished Training
Estimated all
RMSE: 0.8094
MAE:  0.6202


### With forecasting

In [0]:
def forecast_model_constructor(train_df, test_df, forecaster):
  pytorch_model_ratings = forecaster.ratings[['userId', 'movieId', 'y_dev']].copy()
  pytorch_model_ratings['y_dev'] = pytorch_model_ratings['y_dev'] + 4.5
  base_model = pytorchSR.NeuralNetworkSR(pytorch_model_ratings, test_df, 20, 20, [256], 0.0, lr=3*1e-3, wd=1e-1, max_time=440.0, batch_size=10000, log_freq=500)
  
  return rcm.MeanDeviationsRecommender(base_model, forecaster, rcm.get_timestamps_dict(test_df))

m = ut.cross_validate_cached(forecast_model_constructor, ['cross3k_3000movies0', 'cross3k_3000movies1', 'cross3k_3000movies2'])

[1,   500] loss: 1.621
[1,  1000] loss: 0.879
[2,   500] loss: 0.781
[2,  1000] loss: 0.768
[3,   500] loss: 0.751
[3,  1000] loss: 0.742
[4,   500] loss: 0.711
[4,  1000] loss: 0.710
[5,   500] loss: 0.691
[5,  1000] loss: 0.696
[6,   500] loss: 0.676
[6,  1000] loss: 0.683
[7,   500] loss: 0.665
[7,  1000] loss: 0.675
[8,   500] loss: 0.657
[8,  1000] loss: 0.669
[9,   500] loss: 0.651
[9,  1000] loss: 0.665
[10,   500] loss: 0.646
[10,  1000] loss: 0.661
[11,   500] loss: 0.643
[11,  1000] loss: 0.659
[12,   500] loss: 0.640
[12,  1000] loss: 0.656
[13,   500] loss: 0.636
[13,  1000] loss: 0.654
[14,   500] loss: 0.635
[14,  1000] loss: 0.653
[15,   500] loss: 0.633
[15,  1000] loss: 0.652
[16,   500] loss: 0.633
[16,  1000] loss: 0.651
[17,   500] loss: 0.632
[17,  1000] loss: 0.650
[18,   500] loss: 0.631
[18,  1000] loss: 0.650
[19,   500] loss: 0.629
[19,  1000] loss: 0.650
[20,   500] loss: 0.628
[20,  1000] loss: 0.650
Finished Training
Estimated all
RMSE: 0.8092
MAE:  0.6180


## SVD Algorithm

### Without forecasting

In [0]:
def svd_model_constructor(train_df, test_df, forecaster):
  return surprise.SVD(n_factors=20, n_epochs=20, biased=True)
  
ut.cross_validate_cached(svd_model_constructor, ['cross3k_3000movies0', 'cross3k_3000movies1', 'cross3k_3000movies2'])

RMSE: 0.8103
MAE:  0.6235
FCP:  0.7439
RMSE: 0.8107
MAE:  0.6236
FCP:  0.7434
RMSE: 0.8106
MAE:  0.6240
FCP:  0.7437
Average RMSE = 0.8105609436841816
Average MAE = 0.62369723037497
Average NDPM = 0.29582077352402947
Average FCP = 0.7436683689664833


### With forecasting

In [0]:
def svd_forecast_model_constructor(train_df, test_df, forecaster):
  base_model = surprise.SVD(n_factors=20, n_epochs=20, biased=True)
  return rcm.MeanDeviationsRecommender(base_model, forecaster, rcm.get_timestamps_dict(test_df))

m = ut.cross_validate_cached(svd_forecast_model_constructor, ['cross3k_3000movies0', 'cross3k_3000movies1', 'cross3k_3000movies2'])

RMSE: 0.8015
MAE:  0.6131
FCP:  0.7482
RMSE: 0.8018
MAE:  0.6132
FCP:  0.7480
RMSE: 0.8022
MAE:  0.6139
FCP:  0.7477
Average RMSE = 0.8017988563828841
Average MAE = 0.6133852903098534
Average NDPM = 0.2861783127917851
Average FCP = 0.747965248282196


## Probabilistic Matrix Factorization

### Without forecasting

In [0]:
def pmf_model_constructor(train_df, test_df, forecaster):
  return surprise.SVD(n_factors=20, n_epochs=20, biased=False)
  
ut.cross_validate_cached(pmf_model_constructor, ['cross3k_3000movies0', 'cross3k_3000movies1', 'cross3k_3000movies2'])

RMSE: 0.8360
MAE:  0.6474
FCP:  0.7353
RMSE: 0.8375
MAE:  0.6485
FCP:  0.7338
RMSE: 0.8367
MAE:  0.6483
FCP:  0.7346
Average RMSE = 0.8367430548042759
Average MAE = 0.6480531658924386
Average NDPM = 0.29793424190231854
Average FCP = 0.73458859865642


### With forecasting

In [0]:
def pmf_forecast_model_constructor(train_df, test_df, forecaster):
  base_model = surprise.SVD(n_factors=20, n_epochs=20, biased=False)
  return rcm.MeanDeviationsRecommender(base_model, forecaster, rcm.get_timestamps_dict(test_df))

m = ut.cross_validate_cached(pmf_forecast_model_constructor, ['cross3k_3000movies0', 'cross3k_3000movies1', 'cross3k_3000movies2'])

RMSE: 0.8280
MAE:  0.6380
FCP:  0.7373
RMSE: 0.8293
MAE:  0.6390
FCP:  0.7367
RMSE: 0.8280
MAE:  0.6384
FCP:  0.7377
Average RMSE = 0.8284177726871941
Average MAE = 0.6384743082903097
Average NDPM = 0.2889346719208716
Average FCP = 0.7371901109780253


## Baseline Algorithm

### Without forecasting

In [0]:
def baseline_model_constructor(train_df, test_df, forecaster):
  return surprise.prediction_algorithms.baseline_only.BaselineOnly()
  
ut.cross_validate_cached(baseline_model_constructor, ['cross3k_3000movies0', 'cross3k_3000movies1', 'cross3k_3000movies2'])

Estimating biases using als...
RMSE: 0.8593
MAE:  0.6605
FCP:  0.7109
Estimating biases using als...
RMSE: 0.8594
MAE:  0.6603
FCP:  0.7103
Estimating biases using als...
RMSE: 0.8588
MAE:  0.6604
FCP:  0.7108
Average RMSE = 0.8592043313324186
Average MAE = 0.6603866718593452
Average NDPM = 0.3156578810859119
Average FCP = 0.7106698924439194


### With forecasting

In [0]:
def baseline_forecast_model_constructor(train_df, test_df, forecaster):
  base_model = surprise.prediction_algorithms.baseline_only.BaselineOnly()
  return rcm.MeanDeviationsRecommender(base_model, forecaster, rcm.get_timestamps_dict(test_df))
  
ut.cross_validate_cached(baseline_forecast_model_constructor, ['cross3k_3000movies0', 'cross3k_3000movies1', 'cross3k_3000movies2'])

Estimating biases using als...
RMSE: 0.8565
MAE:  0.6582
FCP:  0.7126
Estimating biases using als...
RMSE: 0.8566
MAE:  0.6580
FCP:  0.7121
Estimating biases using als...
RMSE: 0.8561
MAE:  0.6582
FCP:  0.7123
Average RMSE = 0.8563959678800459
Average MAE = 0.6581581453130875
Average NDPM = 0.3118198377044134
Average FCP = 0.712337747764859


# Split by time point method

In [0]:
def test_model(model, train_dataset, test_dataset):
  
  model.fit(train_dataset)
  predictions = model.test(test_dataset)
  
  print("NDPM : {}".format(evl.NDPM(predictions))) 
  surprise.accuracy.rmse(predictions)
  surprise.accuracy.mae(predictions)
  surprise.accuracy.fcp(predictions)
  
  return predictions

In [0]:
train_df, test_df = ds.split_dataset(movielens20m.ratings, offset_ratio=0.0, train_ratio=0.75, test_ratio=0.25, ratings_order='chrono')

most_popular_movies = ut.get_popular_movies_ids(train_df, movielens20m, 2000)

forecaster = rcm.RatingsForecaster(ft.LastSample)
forecaster.compute_devs(train_df, most_popular_movies.movieId.tolist())
forecaster.precompute_means(test_df)

train_dataset = ds.to_surprise_trainset(train_df)
test_dataset = ds.to_surprise_testset(test_df)
test_timestamps_dict = ut.get_timestamps_dict(test_df)

In [0]:
test_timestamps_dict = ut.get_timestamps_dict(test_df)

## Baseline algortithm

### Without forecasting

In [0]:
algo = surprise.prediction_algorithms.baseline_only.BaselineOnly()
test_model(algo, train_dataset, test_dataset)

Estimating biases using als...


### With forecasting

In [0]:
algo = surprise.prediction_algorithms.baseline_only.BaselineOnly()
f_algo = rcm.MeanDeviationsRecommender(algo, forecaster, test_timestamps_dict)

test_model(f_algo, train_dataset, test_dataset)

Estimating biases using als...
NDPM : 0.35472955774690285
RMSE: 0.9358
MAE:  0.7153
FCP:  0.6253


In [0]:
importlib.reload(rcm)
importlib.reload(ft)
importlib.reload(ds)
importlib.reload(pytorchSR)
importlib.reload(ut)
algo = surprise.prediction_algorithms.baseline_only.BaselineOnly()
f_algo = rcm.MeanDeviationsRecommender(algo, forecaster, test_timestamps_dict, True)

test_model(f_algo, train_dataset, test_dataset)

Estimating biases using als...
NDPM : 0.5
RMSE: 1.7606
MAE:  1.4358
FCP:  0.0000


[Prediction(uid=29882, iid=1653, r_ui=3.5, est=5.0, details={'was_impossible': False}),
 Prediction(uid=29882, iid=1639, r_ui=3.0, est=5.0, details={'was_impossible': False}),
 Prediction(uid=29882, iid=1584, r_ui=3.0, est=5.0, details={'was_impossible': False}),
 Prediction(uid=29882, iid=1552, r_ui=3.5, est=5.0, details={'was_impossible': False}),
 Prediction(uid=29882, iid=1485, r_ui=2.0, est=5.0, details={'was_impossible': False}),
 Prediction(uid=29882, iid=1407, r_ui=2.0, est=5.0, details={'was_impossible': False}),
 Prediction(uid=29882, iid=1387, r_ui=3.5, est=5.0, details={'was_impossible': False}),
 Prediction(uid=29882, iid=1370, r_ui=3.0, est=5.0, details={'was_impossible': False}),
 Prediction(uid=29882, iid=1269, r_ui=0.5, est=5.0, details={'was_impossible': False}),
 Prediction(uid=29882, iid=1213, r_ui=4.5, est=5.0, details={'was_impossible': False}),
 Prediction(uid=29882, iid=1207, r_ui=4.0, est=5.0, details={'was_impossible': False}),
 Prediction(uid=29882, iid=1089,

## SVD Algorithm

### Without forecasting

In [0]:
algo = surprise.SVD(n_factors=20, n_epochs=20, biased=True)
non_f_preds = test_model(algo, train_dataset, test_dataset)

NDPM : 0.3524476174094267
RMSE: 0.9310
MAE:  0.7139
FCP:  0.6265


### With forecasting

In [0]:
algo = surprise.SVD(n_factors=20, n_epochs=20, biased=True)
f_algo = rcm.MeanDeviationsRecommender(algo, forecaster, test_timestamps_dict)

preds = test_model(f_algo, train_dataset, test_dataset)

NDPM : 0.35253238898739103
RMSE: 0.9316
MAE:  0.7114
FCP:  0.6282


## Probabilistic Matrix Factorization

### Without forecasting

In [0]:
algo = surprise.SVD(n_factors=20, n_epochs=10, biased=False)
test_model(algo, train_dataset, test_dataset)

NDPM : 0.4837062061817291
RMSE: 1.0097
MAE:  0.7783
FCP:  0.4098

### With forecasting

In [0]:
algo = surprise.SVD(n_factors=20, n_epochs=10, biased=False)
f_algo = rcm.MeanDeviationsRecommender(algo, forecaster, test_timestamps_dict)

test_model(f_algo, train_dataset, test_dataset)

NDPM : 0.3545251071714166
RMSE: 0.9426
MAE:  0.7192
FCP:  0.6231

## Neural Network algorithm

### Without forecasting

In [0]:
algo = pytorchSR.NeuralNetworkSR(train_df, test_df, 1, 20, [256], 0.0, lr=3*1e-3, wd=1e-1, max_time=440.0, batch_size=10000, log_freq=500)
test_model(algo, train_dataset, test_dataset)

### With forecasting

In [0]:
pytorch_model_ratings = forecaster.ratings[['userId', 'movieId', 'y_dev']].copy()
pytorch_model_ratings['y_dev'] = pytorch_model_ratings['y_dev'] + 4.5
algo = pytorchSR.NeuralNetworkSR(pytorch_model_ratings, test_df, 20, 20, [256], 0.0, lr=3*1e-3, wd=1e-1, max_time=440.0, batch_size=10000, log_freq=500)
f_algo = rcm.MeanDeviationsRecommender(algo, forecaster, test_timestamps_dict)

test_model(f_algo, train_dataset, test_dataset)

# Popularity decay split method

In [0]:
# drop 80 % ratings from timebins > 1
filtered = ds.filter_dataset_for_forecasting(movielens20m.ratings, timebin=1)

In [0]:
train_df, test_df = ds.split_dataset_for_forecasting(movielens20m.ratings)
test_df = test_df[test_df.timebin > 3]

train_dataset = ds.to_surprise_trainset(train_df)
test_dataset = ds.to_surprise_testset(test_df)

In [0]:
most_popular_movies = ut.get_popular_movies_ids(train_df, movielens20m, 2000)

forecaster = rcm.RatingsForecaster(ft.LastSample)
forecaster.compute_devs(train_df, most_popular_movies.movieId.tolist())
forecaster.precompute_means(test_df)
test_timestamps_dict = ut.get_timestamps_dict(test_df)

## Baseline algortithm

### Without forecasting

In [0]:
algo = surprise.prediction_algorithms.baseline_only.BaselineOnly()

p = test_model(algo, train_dataset, test_dataset)

Estimating biases using als...
NDPM : 0.3281957041447908
RMSE: 0.8556
MAE:  0.6429
FCP:  0.6620


### With forecasting

In [0]:
algo = surprise.prediction_algorithms.baseline_only.BaselineOnly()
f_algo = rcm.MeanDeviationsRecommender(algo, forecaster, test_timestamps_dict)

p = test_model(f_algo, train_dataset, test_dataset)

Estimating biases using als...
NDPM : 0.32279605820034374
RMSE: 0.8512
MAE:  0.6424
FCP:  0.6652


## SVD Algorithm

### Without forecasting

In [0]:
algo = surprise.SVD(n_factors=20, n_epochs=20, biased=True)

p = test_model(algo, train_dataset, test_dataset)

NDPM : 0.31160740961356553
RMSE: 0.8178
MAE:  0.6139
FCP:  0.6818


### With forecasting

In [0]:
algo = surprise.SVD(n_factors=20, n_epochs=20, biased=True)
f_algo = rcm.MeanDeviationsRecommender(algo, forecaster, test_timestamps_dict)

p = test_model(f_algo, train_dataset, test_dataset)

NDPM : 0.3094253715893138
RMSE: 0.8179
MAE:  0.6143
FCP:  0.6823


## Probabilistic Matrix Factorization

### Without forecasting

In [0]:
algo = surprise.SVD(n_factors=20, n_epochs=20, biased=False)

p = test_model(algo, train_dataset, test_dataset)

NDPM : 0.30841340544582035
RMSE: 0.8553
MAE:  0.6337
FCP:  0.6809


### With forecasting

In [0]:
algo = surprise.SVD(n_factors=20, n_epochs=20, biased=False)
f_algo = rcm.MeanDeviationsRecommender(algo, forecaster, test_timestamps_dict)

p = test_model(f_algo, train_dataset, test_dataset)

NDPM : 0.31017355874108393
RMSE: 0.8791
MAE:  0.6498
FCP:  0.6725


## Neural Network algorithm

### Without forecasting

In [0]:
algo = pytorchSR.NeuralNetworkSR(train_df, test_df, 20, 20, [256], 0.0, lr=3*1e-3, wd=1e-1, max_time=7000.0, batch_size=10000, log_freq=500)
p = test_model(algo, train_dataset, test_dataset)

[1,   500] loss: 1.710
[2,   500] loss: 0.842
[3,   500] loss: 0.799
[4,   500] loss: 0.766
[5,   500] loss: 0.740
[6,   500] loss: 0.722
[7,   500] loss: 0.707
[8,   500] loss: 0.696
[9,   500] loss: 0.688
[10,   500] loss: 0.681
[11,   500] loss: 0.675
[12,   500] loss: 0.669
[13,   500] loss: 0.664
[14,   500] loss: 0.660
[15,   500] loss: 0.658
[16,   500] loss: 0.656
[17,   500] loss: 0.653
[18,   500] loss: 0.651
[19,   500] loss: 0.649
[20,   500] loss: 0.648
Finished Training
Estimated all
NDPM : 0.3060602449250163
RMSE: 0.8177
MAE:  0.6146
FCP:  0.6861


In [0]:
algo = pytorchSR.NeuralNetworkSR(train_df, test_df, 1, 20, [256], 0.0, lr=3*1e-3, wd=1e-1, max_time=7000.0, batch_size=10000, log_freq=500)
p = test_model(algo, train_dataset, test_dataset)

### With forecasting

In [0]:
pytorch_model_ratings = forecaster.ratings[['userId', 'movieId', 'y_dev']].copy()
pytorch_model_ratings['y_dev'] = pytorch_model_ratings['y_dev'] + 6.0
algo = pytorchSR.NeuralNetworkSR(pytorch_model_ratings, test_df, 20, 20, [256], 0.0, lr=3*1e-3, wd=1e-1, max_time=440.0, batch_size=10000, log_freq=500)
f_algo = rcm.MeanDeviationsRecommender(algo, forecaster, test_timestamps_dict)

p = test_model(f_algo, train_dataset, test_dataset)

[1,   500] loss: 1.640
[2,   500] loss: 0.833
[3,   500] loss: 0.791
[4,   500] loss: 0.774
[5,   500] loss: 0.742
[6,   500] loss: 0.725
[7,   500] loss: 0.710
[8,   500] loss: 0.694
[9,   500] loss: 0.684
[10,   500] loss: 0.675
[11,   500] loss: 0.669
[12,   500] loss: 0.663
[13,   500] loss: 0.659
[14,   500] loss: 0.654
[15,   500] loss: 0.652
[16,   500] loss: 0.649
[17,   500] loss: 0.647
[18,   500] loss: 0.645
[19,   500] loss: 0.644
[20,   500] loss: 0.642
Finished Training
Estimated all
NDPM : 0.30359779483382343
RMSE: 0.8162
MAE:  0.6133
FCP:  0.6872


# Time-awareness neural network model

In [0]:
movielens20m = ds.get_movielens('20m')
ut.add_timebins(movielens20m.ratings,10)

### Without time-awareness

In [0]:
def model_constructor(train_df, test_df):
  return pytorchSR.NeuralNetworkSR(train_df, test_df, 20, 20, [256], 0.0, lr=3*1e-3, wd=1e-1, max_time=440.0, batch_size=10000, log_freq=500, time_biased=False)

ut.cross_validate(model_constructor, movielens20m.ratings, k=3)

[1,   500] loss: 1.672
[1,  1000] loss: 0.891
[2,   500] loss: 0.796
[2,  1000] loss: 0.781
[3,   500] loss: 0.749
[3,  1000] loss: 0.738
[4,   500] loss: 0.718
[4,  1000] loss: 0.719
[5,   500] loss: 0.698
[5,  1000] loss: 0.703
[6,   500] loss: 0.682
[6,  1000] loss: 0.690
[7,   500] loss: 0.671
[7,  1000] loss: 0.683
[8,   500] loss: 0.664
[8,  1000] loss: 0.677
[9,   500] loss: 0.658
[9,  1000] loss: 0.673
[10,   500] loss: 0.653
[10,  1000] loss: 0.670
[11,   500] loss: 0.649
[11,  1000] loss: 0.666
[12,   500] loss: 0.646
[12,  1000] loss: 0.664
[13,   500] loss: 0.644
[13,  1000] loss: 0.663
[14,   500] loss: 0.642
[14,  1000] loss: 0.661
[15,   500] loss: 0.641
[15,  1000] loss: 0.661
[16,   500] loss: 0.640
[16,  1000] loss: 0.659
[17,   500] loss: 0.639
[17,  1000] loss: 0.658
[18,   500] loss: 0.637
[18,  1000] loss: 0.658
[19,   500] loss: 0.637
[19,  1000] loss: 0.657
[20,   500] loss: 0.636
[20,  1000] loss: 0.657
Finished Training
Estimated all
RMSE: 0.8099
MAE:  0.6200


### With forecasting

### Cross validation

In [0]:
def model_constructor(train_df, test_df):
  return pytorchSR.NeuralNetworkSR(train_df, test_df, 20, 20, [256], 0.0, lr=3*1e-3, wd=1e-1, max_time=440.0, batch_size=10000, log_freq=500, time_biased=True, time_bins=51)

ut.cross_validate(model_constructor, movielens20m.ratings, k=3)

[1,   500] loss: 2.215
[1,  1000] loss: 1.052
[2,   500] loss: 0.813
[2,  1000] loss: 0.782
[3,   500] loss: 0.754
[3,  1000] loss: 0.740
[4,   500] loss: 0.712
[4,  1000] loss: 0.713
[5,   500] loss: 0.692
[5,  1000] loss: 0.697
[6,   500] loss: 0.677
[6,  1000] loss: 0.686
[7,   500] loss: 0.666
[7,  1000] loss: 0.678
[8,   500] loss: 0.660
[8,  1000] loss: 0.673
[9,   500] loss: 0.655
[9,  1000] loss: 0.670
[10,   500] loss: 0.651
[10,  1000] loss: 0.667
[11,   500] loss: 0.647
[11,  1000] loss: 0.664
[12,   500] loss: 0.644
[12,  1000] loss: 0.662
[13,   500] loss: 0.642
[13,  1000] loss: 0.660
[14,   500] loss: 0.640
[14,  1000] loss: 0.659
[15,   500] loss: 0.638
[15,  1000] loss: 0.658
[16,   500] loss: 0.637
[16,  1000] loss: 0.656
[17,   500] loss: 0.636
[17,  1000] loss: 0.656
[18,   500] loss: 0.635
[18,  1000] loss: 0.655
[19,   500] loss: 0.633
[19,  1000] loss: 0.655
[20,   500] loss: 0.633
[20,  1000] loss: 0.654
Finished Training
Estimated all
RMSE: 0.8091
MAE:  0.6195


In [0]:
# drop 80 % ratings from timebins > 1
filtered = ds.filter_dataset_for_forecasting(movielens20m.ratings, timebin=1)

In [0]:
train_df, test_df = ds.split_dataset_for_forecasting(movielens20m.ratings)
test_df = test_df[test_df.timebin > 3]

In [0]:
test_df.userId.isin(train_df.userId).value_counts()

True     367608
False       209
Name: userId, dtype: int64

In [0]:
train_dataset = ds.to_surprise_trainset(train_df)
test_dataset = ds.to_surprise_testset(test_df)

In [0]:
most_popular_movies = ut.get_popular_movies_ids(train_df, movielens20m, 2000)

forecaster = rcm.RatingsForecaster(ft.LastSample)
forecaster.compute_devs(train_df, most_popular_movies.movieId.tolist())
forecaster.precompute_means(test_df)
test_timestamps_dict = ut.get_timestamps_dict(test_df)

## Baseline algortithm

### Without forecasting

In [0]:
algo = surprise.prediction_algorithms.baseline_only.BaselineOnly()

p = test_model(algo, train_dataset, test_dataset)

Estimating biases using als...
NDPM : 0.3281957041447908
RMSE: 0.8556
MAE:  0.6429
FCP:  0.6620


### With forecasting

In [0]:
algo = surprise.prediction_algorithms.baseline_only.BaselineOnly()
f_algo = rcm.MeanDeviationsRecommender(algo, forecaster, test_timestamps_dict)

p = test_model(f_algo, train_dataset, test_dataset)

Estimating biases using als...
NDPM : 0.32279605820034374
RMSE: 0.8512
MAE:  0.6424
FCP:  0.6652
